### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [2]:
%sh aql -c "truncate test" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>7.1.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
import com.aerospike.client.policy.Policy;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [6]:
Key key = new Key("test", "demo", "key1");
WritePolicy wPolicy = new WritePolicy();
Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));
client.put(wPolicy, key, b1, b2);
Record record = client.get(null, key);
System.out.println("Record values are:");
System.out.println(record);

Record values are:
(gen:1),(exp:433899090),(bins:(b1:val1),(b2:2))


In [7]:
// convenience function to create an index - essentially a pass-through to the client API
void myCreateIndex(String nsName, String setName, String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    Policy policy = new Policy();
    try {
            IndexTask task = client.createIndex(policy, nsName, setName, idxName, binName, idxType, colType, ctx);
            task.waitTillComplete(100, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
        System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
                                    idxName, nsName, setName, binName);
}
void myDropIndex(String nsName, String setName, String idxName) {
    try {
        IndexTask task = client.dropIndex(null, nsName, setName, idxName);
    }
    catch (AerospikeException ae) {
        if (ae.getResultCode() != ResultCode.INDEX_NOTFOUND) {
            throw ae;
        }
    } 
    System.out.format("Dropped index %s.\n", idxName);
}

In [9]:
String nsName = "test";
for(int i=0; i<1026; i++){ 
  String idxname = "name"+i;
  String binname = "bin"+i;
  String setName = "set"+i;
  myCreateIndex(nsName, setName, idxname, binname, IndexType.NUMERIC, IndexCollectionType.DEFAULT);
  myDropIndex(nsName, setName, idxname);
}

Created index name0 on ns=test set=set0 bin=bin0.
Created index name1 on ns=test set=set1 bin=bin1.
Created index name2 on ns=test set=set2 bin=bin2.
Created index name3 on ns=test set=set3 bin=bin3.
Created index name4 on ns=test set=set4 bin=bin4.
Created index name5 on ns=test set=set5 bin=bin5.
Created index name6 on ns=test set=set6 bin=bin6.
Created index name7 on ns=test set=set7 bin=bin7.
Created index name8 on ns=test set=set8 bin=bin8.
Created index name9 on ns=test set=set9 bin=bin9.
Created index name10 on ns=test set=set10 bin=bin10.
Created index name11 on ns=test set=set11 bin=bin11.
Created index name12 on ns=test set=set12 bin=bin12.
Created index name13 on ns=test set=set13 bin=bin13.
Created index name14 on ns=test set=set14 bin=bin14.
Created index name15 on ns=test set=set15 bin=bin15.
Created index name16 on ns=test set=set16 bin=bin16.
Created index name17 on ns=test set=set17 bin=bin17.
Created index name18 on ns=test set=set18 bin=bin18.
Created index name19 on

In [11]:
Key key = new Key("test", "demo3", "key1");  //New set name
WritePolicy wPolicy = new WritePolicy();
Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));
client.put(wPolicy, key, b1, b2);
Record record = client.get(null, key);
System.out.println("Record values are:");
System.out.println(record);

EvalException: Error 4,1,0,30000,1000,0,A1 127.0.0.1 3000: Parameter error